<a href="https://colab.research.google.com/github/jhermosillo/diplomado_CDD2019/blob/master/04%20Clasificacion_Agrupamiento/2%20Modelos%20Probabilistas/Notebooks/Naive_Bayes_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="font-size:50px;" align="left"> <img align="left" width="400" src="../img/cerebro_1.jpg"/>Clasificador Naïve Bayes<br>
    Detector de Spam

In [2]:
import pandas as pd
import numpy as np

## Archivo de datos preprocesados

### Matriz BOW

* Los correos vienen etiquetados de la siguiente forma: `correo_enron1_<0/1>_<#email>` donde $0\equiv Ham$ y $1\equiv Spam$.

In [3]:
data = pd.read_csv('../Data/frequencia_palabras.csv',index_col=0) # lectura del documento con pandas
# visualizacion
print(data.shape)
data.head()

(9406, 1001)


,WORD,correo_enron1_0_0,correo_enron1_0_1,correo_enron1_0_2,correo_enron1_0_3,correo_enron1_0_4,correo_enron1_0_5,correo_enron1_0_6,correo_enron1_0_7,correo_enron1_0_8,...,correo_enron1_1_490,correo_enron1_1_491,correo_enron1_1_492,correo_enron1_1_493,correo_enron1_1_494,correo_enron1_1_495,correo_enron1_1_496,correo_enron1_1_497,correo_enron1_1_498,correo_enron1_1_499
0,fw,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,everyone,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,access,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,change,1,0,0,0,5,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,security,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


La siguiente función separa los datos para prueba y entrenamiento, el argumento 'resto' indica la cantidad de correos que se desea para los datos de prueba. <br>
La salida son dos diccionarios, uno de entrenamiento y el otro para prueba del modelo.

In [4]:
import random
import math

def data_split(data,test=0.2):
    pos_cols=[col for col in data.columns if 'correo_enron1_1' in col]
    neg_cols=[col for col in data.columns if 'correo_enron1_0' in col]
    
    pos_train=random.sample(pos_cols,k=math.floor(len(pos_cols)*(1-test)))
    neg_train=random.sample(neg_cols,k=math.floor(len(neg_cols)*(1-test)))
    
    columnas=neg_train + pos_train
    x_train=data.loc[:,columnas].T

    pos_test=[item for item in pos_cols if item not in pos_train]
    neg_test=[item for item in neg_cols if item not in neg_train]
    
    columnas=neg_test + pos_test
    x_test=data.loc[:,columnas].T
    
    y_test_pos=np.ones(len(pos_test)).astype(int)
    y_test_neg=np.zeros(len(neg_test)).astype(int)
    y_test=np.concatenate((y_test_neg,y_test_pos))

    return x_train,x_test,y_test

In [20]:
X_train, X_test, y_test = data_split(data)
print('Correos de Entrenamiento: {0} ; Features (palabras): {1}'
      .format(X_train.shape[0],X_train.shape[1]))
print('Correos de Prueba: \t  {0} ; Features (palabras): {1}'
      .format(X_test.shape[0],X_test.shape[1]))
print('Clases de Prueba: {0} '.format(y_test))
X_test.head()

Correos de Entrenamiento: 800 ; Features (palabras): 9406
Correos de Prueba: 	  200 ; Features (palabras): 9406
Clases de Prueba: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] 


,0,1,2,3,4,5,6,7,8,9,...,9396,9397,9398,9399,9400,9401,9402,9403,9404,9405
correo_enron1_0_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
correo_enron1_0_7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
correo_enron1_0_25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
correo_enron1_0_41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
correo_enron1_0_44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Entrenamiento - NAIVE BAYES MULTINOMIAL
La distribución de probabilidad de la Variable Aleatoria $\textrm{x}$ es una distribución multinomial (conteos de palabras) de la conjunción de palabras únicas (vocabulario-$Voc$).<br>

### &#9998; Entrena al modelo utilizando la siguiente fórmula:

$$ P(x_i|y_j) = \theta_i^j=\frac{n_i + 1}{|S_j| + k} ;\;\;\;i=1,2,\ldots,k \;; j\in\{Spam,Ham\}$$ .

* $|S_j|$ es la suma de los conteos de cada palabra para la clase $y_j$. 
* $k=|Voc|$.
* $n_i$ es la suma de los conteos para la palabra $i$.

&#9758; **Crea una función que reciba un arreglo o un dataframe $X_{train}$ y devuelva dos vectores: $\theta_p$ y $\theta_n$.**
* Recuerda que $\theta_p=(\theta_1^{Spam},\ldots,\theta_k^{Spam})$ y $\theta_n=(\theta_1^{Ham},\ldots,\theta_k^{Ham})$

In [10]:
def fit(X_train):
    """
    Entrenamiento del clasificador NAIVE BAYES:
    Entrada: 
     - X_train: un DataFrame o Arreglo con instancias de entrenamiento.
    Salida:
     - una lista: [theta_n,theta_p] conteniendo 
       los vectores (np.arrays) de probabilidad de cada feature (palabra) 
       en correos Ham y Spam respectivamente. 
    """
    k = X_train.shape[1] # numero de palabras
    print('numero de palabras: ',k)
    correos = X_train.shape[0]
    print("número de correos: ", correos)
        
    #entranamiento para Ham y Spam
    """ TU CODIGO AQUI"""    

    
    
    """ ------------- """
    print('theta shapes=',theta[0].shape,theta[1].shape)#debe ser (k,) para ambos

    return theta[0],theta[1]


&#9758; **Entrena el modelo**

In [11]:
theta_n, theta_p=fit(X_train)

numero de palabras:  9406
número de correos:  800
theta shapes= (9406,) (9406,)


In [12]:
print("La suma para vector debe ser 1 \n Theta_p : ",np.sum(theta_p))
print("La suma para vector debe ser 1 \n Theta_n : ",np.sum(theta_n))

La suma para vector debe ser 1 
 Theta_p :  1.0
La suma para vector debe ser 1 
 Theta_n :  1.0


## Prueba del Modelo

#### &#9998; Define la función que calcula la verosimilitud:<br> <br>

$$ p(\textrm{x}|\theta) =  \prod_{i=1}^{k} p(x_i|\theta) = n!\times\frac{\theta_{1}^{x_1}}{x_1!}\times...\times\frac{\theta_k^{x_k}}{x_k!}$$ <br>
donde $n=\sum x_i$.

* **NOTA**: debido a que estamos trabajando con números muy pequeños de probablidad, se sugiere calcular el $\textrm{log}(P(x|theta))$ para después utilizar el criterio de decisión de Likelyhood Ratio (LR).
* Se recomienda usar:
  - `np.math.log` para escalares
  - `np.log` para arreglos
  - `np.math.factorial` para escalares 
  - `scipy.special.factorial` para arreglos.
  - `np.math.exp` como inverso de `log`

In [13]:
from scipy.special import factorial

def P_Mi(x,theta):
    """
    Cálcula la función de verosimilitud: P(x|theta)
    Entrada:
     - x: arreglo unidimensional de datos
     - theta: arreglo unidimensional de probabilidades dim(theta)=dim(x)
    Salida:
     - Valor de log(P(x|theta)) según distribución MULTINOMIAL de x
    """
    P_x=0
    """ TU CODIGO AQUI"""      

    
    
    """ ------------- """
    
    return np.math.exp(P_x)

#### &#9998; Define la función que clasifica X mediante el criterio LR:<br> <br>

$$ LR = \frac{p(\textrm{x}|\theta_{Pos})}{p(\textrm{x}|\theta_{Neg})}$$ <br>

Este modelo utiliza la razón de verosimilitud (LR) para hacer la predición.<br>
Si LR > 1 -> y=1 <br>
Si LR < 1 -> y=0 <br>

In [15]:
def clasifica_LR(X_test,theta_p, theta_n):
    """
    Calcula: LR=P(x|y=+)/P(x|y=-)
    Entrada:
     - X_test: arreglo o Data Frame de datos
     - theta_p: arreglo unidimensional de probabilidades Pos
     - theta_n: arreglo unidimensional de probabilidades Neg
    Salida:
     - Valor de LR
    """
    instancias = X_test.shape[0]
    y = np.zeros(instancias)
    print(y.shape)
    """ TU CODIGO AQUI """

    
    
    
    """ -------------  """
    return y.astype(int)    

## Test del modelo
La siguiente función realiza el test del conjunto de pruebas.<br>
Recibe el arreglo de datos de prueba, el vector theta positivo y negativo para el LR.

In [ ]:
y_predic = clasifica_LR(X_test,theta_p, theta_n)
print(y_predic)

## Matriz de confusión

In [17]:
def matrizCnf(y_test,y_predic):
    totalInstancias = y_test.shape[0]
    tNegativo = len(np.where(y_predic[0:100]==y_test[0:100])[0]) # true negative Los primeros 100 son correos ham
    fNegativo = len(np.where(y_predic[0:100]!=y_test[0:100])[0]) # false negative Los primeros 100 son correos ham
    #print(tPositivo,fPositivo)
    tPositivo = len(np.where(y_predic[100:200]==y_test[100:200])[0]) # true positive Los siguientes 100 son correos spam
    fPositivo = len(np.where(y_predic[100:200]!=y_test[100:200])[0]) # false positive Los siguientes 100 son correos spam
    #print(tNegativo,fNegativo)
    matriz = {'0':["Real: negativo","Real: positivo"]}
    matriz["prediccion: negativo"] = [tNegativo,fNegativo]
    matriz["prediccion: positivo"] = [fPositivo,tPositivo]
    return matriz

In [18]:
matrizConfusion = matrizCnf(y_test,y_predic)

In [ ]:
pd.DataFrame(matrizConfusion).head()